<a href="https://colab.research.google.com/github/leandroaguazaco/data_science_portfolio/blob/main/Projects/04-Churn_Telco_Analysis/04_Churn_Telco_Analysis_01_Preprocessing_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> 4 - CHURN TELCO ANALYSIS </h1>
<h2 align="center"> 4.1 - Preprocessing </h2>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_project-true-blue">

  <img alt="Static Badge" src="https://img.shields.io/badge/status-in progress-green">

</div>  

<object
data="https://img.shields.io/badge/contact-Felipe_Leandro_Aguazaco-blue?style=flat&link=https%3A%2F%2Fwww.linkedin.com%2Fin%2Ffelipe-leandro-aguazaco%2F">
</object>

## a. Project summary

The aim of this project is to analyze and predict customer churn in the telco industry. The information pertains to client behavior, including in-call, out-call, and internet service consumption. There is a variable called 'Churn' that determines whether a customer churned within two weeks after canceling services. The information summarizes eight weeks of data for each telco line or client.

<h3 align="center"> <font color='orange'>NOTE: The project is distributed across multiple sections, separated into notebook files, in the following way:</font> </h3>



> <font color='gray'> 4.1 - Preprocessig data: load, join and clean data, and Exploratory data analysis, EDA.</font> ✍ ▶ Current section

4.2 - Pre-modeling: predict customer churn based on PyCaret library.

4.3 - Modeling: predict customer churn based on sklearn pipelines.

4.4 - Analyzing and explaining predictions.

4.5 - Detecting vulneabilities in final machine learnig model.

4.6 - Model deployment with Streamlit.

## b. Install libraries

Additional libraries such as pandas, numpy, matplotlib, seaborn, and others are already installed in the Colab environment.

In [1]:
%%capture
!pip install pandas
!pip install polars
!pip install xlsx2csv
!pip install pyjanitor # Clean DataFrame
!pip install colorama
!pip install adjustText
!pip install rpy2==3.5.1 # Use R

## c. Import libraries

In [189]:
%%capture
# c.1 Python Utilies
import pandas as pd
import polars as pl
import numpy as np
import functools
import glob
import math
from scipy.stats import spearmanr
import scipy.stats as stats
import warnings
from janitor import clean_names, remove_empty
import rpy2
import shutil
from google.colab import drive
import os

# c.2 Visulization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from colorama import Fore, Style

In [ ]:
# c.3 Setups
%matplotlib inline
plt.style.use("ggplot")
warnings.simplefilter("ignore")

## d. Custom functions

### d.1 Load csv files

In [156]:
def custom_readcsv(filepath: str = None) -> pd.DataFrame:
  """
  Summary:
    Function to read a csv files, set SUBSCRIBER_ID as index column and drop unncessary column.
  Parameters:
    file(str, default = None): path to your file of interest.
  Return
    pandas DataFrame.
  """
  df = pd.read_csv(filepath_or_buffer = filepath,
                   sep = "|",
                   index_col = "SUBSCRIBER_ID",
                   parse_dates = True,
                   decimal = ",",
                   encoding = "utf-8") \
         .pipe(lambda x: x.drop([x.columns[0]], axis = 1)) \
         .pipe(clean_names)

  return df

### d.2 Type conversions

In [5]:
# d.1 dtypes conversion and memory reduce function.
def dtype_conversion(df: pd.DataFrame = None, verbose: bool = True)-> pd.DataFrame:
    """
    Summary:
      Function to dtypes conversion and save reduce memory usage; takes a DataFrame as argument, returns DataFrame.
      For more details, visit: https://towardsdatascience.com/how-to-work-with-million-row-datasets-like-a-pro-76fb5c381cdd.
      The modifications include type casting for numerical and object variables.
    Parameters:
      df (pandas.DataFrame): DataFrame containing information.
      verbose (bool, default = True): If true, display results (conversions and warnings)
    Returns:
      pandas.DataFrame: original DataFrame with dtypes conversions
      Plot original dtypes status, variable warning due high cardinality, save memory usage, final dtypes status.
    """
    # 0- Original dtypes
    # print(Fore.GREEN + "Input dtypes" + Style.RESET_ALL)
    # print(df.dtypes)
    # print("\n")
    print(Fore.RED + "High Cardinality, categorical features with levels > 15" + Style.RESET_ALL)

    # 1- Original memory_usage in MB
    start_mem = df.memory_usage().sum() / 1024 ** 2

    # 2- Numerical Types
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int": # First 3 characters
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # 3- Categorical Types
    high_card_vars = 0
    for col in df.select_dtypes(exclude = ["int8", "int16", "int32", "int64", "float16", "float32", "float64", "datetime64[ns]"]):
        categories = list(df[col].unique())
        cat_len = len(categories)
        if cat_len >= 2 and cat_len < 15:
           df[col] = df[col].astype("category")
        else:
          high_card_vars =+ 1
          # Print hight cardinality variables, amount of levels and a sample of 50 firts categories
          print(f"Look at: {Fore.RED + col + Style.RESET_ALL}, {cat_len} levels = {categories[:50]}")
    if high_card_vars == 0:
      print(Fore.GREEN + "None" + Style.RESET_ALL)
    else:
      pass

    # 4- Final memory_usage in MB
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print("\n")
        print(f"{Fore.RED}Initial memory usage: {start_mem:.2f} MB{Style.RESET_ALL}")
        print(f"{Fore.BLUE}Memory usage decreased to {end_mem:.2f} MB ({ 100 * (start_mem - end_mem) / start_mem:.1f}% reduction){Style.RESET_ALL}")
        #print("\n")
        #print(Fore.GREEN + "Output dtypes" + Style.RESET_ALL)
        #print(df.dtypes)
        print("\n")

    # 5. Feature types
    print(Fore.GREEN + "Variable types" + Style.RESET_ALL)
    numerical_vars = len(df.select_dtypes(include = ["number"]).columns)
    categorical_vars = len(df.select_dtypes(include = ["category", "object"]).columns)
    datetime_vars = len(df.select_dtypes(include = ["datetime64[ns]"]).columns)
    print(f"Numerical Features: {numerical_vars}")
    print(f"Categorical Features: {categorical_vars}")
    print(f"Datetime Features: {datetime_vars}")

    return df

## 1 - Load and clean data

### 1.1 - Importing files

Four files (.csv):

* CONSUMO_DATOS.csv
* CONSUMO_VOZ_IN.csv
* CONSUMO_VOZ_OUT.csv
* INFORMACION_GENERAL.csv

In [309]:
# 1. List of .csv files inside Colab content folder
files = glob.glob('/content/' + '/*.csv')
files

['/content/CONSUMO_VOZ_IN.csv',
 '/content/CONSUMO_VOZ_OUT.csv',
 '/content/INFORMACION_GENERAL.csv',
 '/content/CONSUMO_DATOS.csv']

In [310]:
# 2. Read .csv files and save in an array
df_list = [custom_readcsv(file) for file in files]

In [311]:
# 3. Aggregate info from datasets

# Names, rows and columns from datasets
files_numb = np.arange(0, len(df_list))
dicts = [{"name": files[i][files[i].rfind('/') + 1: ], "rows": df_list[i].shape[0], "cols": df_list[i].shape[1]} for i in files_numb]

files_info = pd.DataFrame(dicts) \
               .sort_values(by = ["rows", "name"], ascending = [False, True])

# Reorder original file list according dataset's rows
files = list(map(files.__getitem__, files_info.index))

files_info

,name,rows,cols
2,INFORMACION_GENERAL.csv,793674,13
0,CONSUMO_VOZ_IN.csv,740366,10
3,CONSUMO_DATOS.csv,740364,8
1,CONSUMO_VOZ_OUT.csv,740364,12


In [312]:
# 4. Join the four datasets, in files list, by index (subscriber_id). Drop duplicates
churn_df = functools.reduce(lambda left, right: left.join(right, how = 'left', validate = None), df_list) \
                    .drop_duplicates(keep = "first", ignore_index = False) \
                    .round(3)

# round(3) to reduce the file size

### 1.2 - Tidying categorical data


In [315]:
# Categorical variables
for i in churn_df.select_dtypes(exclude = ["number", "datetime64[ns]"]):
  cat_levels = churn_df[[i]].groupby(by = i, as_index = False).agg(freq = pd.NamedAgg(column = i, aggfunc = "count"))
  cat_levels.rename(columns = {i: "level"}, inplace = True)
  print(f"{Fore.RED}{i}{Style.RESET_ALL}")
  print(cat_levels)
  print("\n")

# Erroneous entries in canal, region and bandas features

canal
                        level    freq
0                     Cadenas   20701
1           Convenios Remotos     148
2                   E-Dealers   19676
3       External organization     207
4   Fuerza De Venta Indirecta  206120
5                     Inbound   53085
6       Internal organization    5923
7                     Kioscos   63455
8                    Outbound  113683
9               Self-employed      20
10                    Tiendas  229968
11            Tiendas Express      88


region
                                          level    freq
0                                      Amazonas     125
1                                     Antioquia   78569
2                                        Arauca     735
3                                     Atlantico   65443
4                                   Bogota D.C.  324292
5                                       Bolivar   17969
6                                        Boyaca    6820
7                                        Ca

In [314]:
# Erroneous entries in canal, region and bandas features
churn_df = churn_df \
           .pipe(pd.DataFrame.replace, {"canal": {"-1": np.nan,
                                                  "-2": np.nan}}) \
           .pipe(pd.DataFrame.replace, {"region": {"-2": np.nan,
                                                   "0": np.nan,
                                                   "Bogota. D.C.": "Bogota D.C.",
                                                   "Nari?O": "Nariño"}}) \
           .pipe(pd.DataFrame.replace, {"bandas": {"NO": "Ninguna",
                                                   "NINGUNA": "Ninguna"}}) \
           .assign(bandas = lambda x: x.loc[:, "bandas"].str.title()) \
           .assign(tipo_gross_adds = lambda x: x.loc[:, "tipo_gross_adds"].str.title())


### 1.3 - Tidying numerical data

In [318]:
# Revisar, según la naturaleza de cada variable, tipo de escala de medida, valores mínimos permitidos
# valores máximos permitos, coherencia en promedio

# porcentajes: valores mínimos >= 0, valores máximos <= 100
# variaciones: pueden tomar valores negativos
# promedios: según naturaleza de la variable y escala de medida, pueden tomar valores negativos y positivos
# conteos: según naturaleza de la variable, valores >= 0

churn_df.select_dtypes(include = "number") \
        .describe() \
        .T \
        .round(3) \
        .sort_index()

,count,mean,std,min,25%,50%,75%,max
cant_sem_datos,714792.0,6.243,2.454,0.000,5.000,8.000,8.000,8.000
churn,713736.0,0.030,0.171,0.000,0.000,0.000,0.000,1.000
cons_ult_sem,714792.0,2.320,3.414,0.000,0.005,0.980,3.210,293.905
contactos_ult_semana_pqr,713736.0,0.121,0.589,0.000,0.000,0.000,0.000,32.000
continuidad_traf,705421.0,0.687,0.373,0.000,0.389,0.615,1.000,2.000
contrafico,714792.0,0.894,0.308,0.000,1.000,1.000,1.000,1.000
jineteo,713736.0,1.025,0.167,1.000,1.000,1.000,1.000,7.000
lineas_cliente,713736.0,1.268,0.720,0.000,1.000,1.000,1.000,46.000
mean_llamadas_in,688220.0,18.432,18.298,1.000,6.000,13.286,24.875,818.250
mean_llamadas_in_num_dif,688220.0,7.467,6.329,1.000,3.000,6.000,10.000,405.125


In [317]:
# Erroneous entries in continuidad_traf feature
churn_df = churn_df \
           .pipe(pd.DataFrame.replace, {"continuidad_traf": {np.inf: np.nan,
                                                             -np.inf: np.nan}}) \
           .assign(continuidad_traf = lambda x: x.loc[:, "continuidad_traf"].astype("float"))


### 1.4 - Final tidying dataset

In [319]:
churn_df = churn_df \
           .pipe(dtype_conversion)

High Cardinality, categorical features with levels > 15
Look at: region, 34 levels = ['Bogota D.C.', 'Santander', 'Antioquia', nan, 'Cundinamarca', 'Quindio', 'Valle Del Cauca', 'Arauca', 'Bolivar', 'Atlantico', 'Tolima', 'Huila', 'Meta', 'Putumayo', 'Boyaca', 'Caldas', 'Cordoba', 'Nariño', 'Magdalena', 'Risaralda', 'Cauca', 'La Guajira', 'Amazonas', 'Norte De Santander', 'Choco', 'Caqueta', 'Cesar', 'Sucre', 'Guaviare', 'Vaupes', 'Guainia', 'Vichada', 'Casanare', 'Providencia Y Santa Catalina (Santa Isabel)']


Initial memory usage: 239.95 MB
Memory usage decreased to 67.49 MB (71.9% reduction)


Variable types
Numerical Features: 39
Categorical Features: 4
Datetime Features: 0


In [320]:
churn_df.head()

,mean_minutos_voz_in,mean_llamadas_in,mean_llamadas_in_num_dif,mean_porcent_llamadas_fallidas_in,mean_porcent_minutos_origen_wom,mean_porcent_minutos_origen_otros,var_minutos_in_4sem,var_llamadas_in_4sem,var_minutos_in_8sem,var_llamadas_in_8sem,...,jineteo,churn,cant_sem_datos,prom_gb_tt,prom_gb_ran,prom_%_propia,continuidad_traf,variacion_datos_8s,cons_ult_sem,contrafico
SUBSCRIBER_ID,,,,,,,,,,,,,,,,,,,,,
192556633,29.515625,5.570312,5.000,0.000000,60.281250,39.75000,1.830078,1.500000,1.398438,1.692383,...,1.0,0.0,1.0,0.603027,0.000000,1.000000,1.000000,0.00000,0.603027,1.0
192557534,52.875000,18.875000,10.875,0.780762,38.718750,61.28125,0.464111,-0.045013,0.290039,-0.151978,...,1.0,0.0,5.0,3.369141,0.075012,0.975098,1.250000,-17.40625,2.783203,1.0
192559244,64.000000,18.875000,9.500,0.000000,15.203125,84.81250,0.325928,0.553223,0.386963,0.748047,...,1.0,0.0,8.0,0.832031,0.018997,0.982910,0.500000,-48.93750,0.425049,1.0
192569904,148.875000,37.000000,18.000,0.000000,5.636719,94.37500,-0.139038,-0.292969,-0.238037,-0.215942,...,1.0,0.0,8.0,6.824219,0.327881,0.933105,0.727051,-60.62500,2.687500,1.0
192574506,77.812500,15.500000,4.000,0.000000,3.138672,96.87500,0.942871,1.208008,2.718750,2.419922,...,1.0,0.0,8.0,0.916992,0.192017,0.810059,0.444092,65.06250,1.513672,1.0


### 1.5 - Export to local txt file

In [322]:
# 5. Export preprocessing result to local file
churn_df.to_csv(path_or_buf = "churn_data.txt",
                sep = "|",
                decimal = ".",
                encoding = "utf-8",
                index_label = "SUBSCRIBER_ID")

### 1.6 - Export to Google Drive

In [326]:
# 6. Export preprocessing result to Google Drive

# Mount Google Drive
drive.mount('/content/drive')

# Specify the temporary file or folder path in Colab
temp_filepath = "/content/churn_data.txt"

# Specify the destination path in Google Drive
drive_filepath = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Remove the existing folder in Google Drive
try:
  shutil.copy(src = temp_filepath, dst = drive_filepath)
except:
  pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1.7 - Import from Google Drive

In [328]:
# Mount Google Drive
drive.mount('/content/drive')


# Specify the source path in Google Drive
drive_filepath = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Specify the destination path in Colab
colab_filepath = '/content/'

# Copy the file from Google Drive to Colab
try:
  shutil.copy(src = drive_filepath + '/churn_data.txt', dst = colab_filepath + '/churn_data.txt')
except:
  pass

churn_df = pd.read_csv(filepath_or_buffer = "churn_data.txt",
                       sep = "|",
                       index_col = "SUBSCRIBER_ID",
                       parse_dates = True,
                       decimal = ".",
                       encoding = "utf-8") \
             .pipe(dtype_conversion)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
High Cardinality, categorical features with levels > 15
Look at: region, 34 levels = ['Bogota D.C.', 'Santander', 'Antioquia', nan, 'Cundinamarca', 'Quindio', 'Valle Del Cauca', 'Arauca', 'Bolivar', 'Atlantico', 'Tolima', 'Huila', 'Meta', 'Putumayo', 'Boyaca', 'Caldas', 'Cordoba', 'Nariño', 'Magdalena', 'Risaralda', 'Cauca', 'La Guajira', 'Amazonas', 'Norte De Santander', 'Choco', 'Caqueta', 'Cesar', 'Sucre', 'Guaviare', 'Vaupes', 'Guainia', 'Vichada', 'Casanare', 'Providencia Y Santa Catalina (Santa Isabel)']


Initial memory usage: 239.95 MB
Memory usage decreased to 67.49 MB (71.9% reduction)


Variable types
Numerical Features: 39
Categorical Features: 4
Datetime Features: 0
